In [30]:
import os, re
from collections import Counter

In [32]:
def preprocess(text):
    """
    Функция для предобработки текста.
    Разбиваем текст на токены, удаляем стоп-слова
    """
    raw = re.split(r'\W+', text.lower())
    with open('nltk_stopwords.txt', 'r') as swf:
        stopwords = swf.read().strip().split(',')
    tokens = []
    for token in raw:
        #если слова нет в стоп-листе и оно длиннее 3 символов, то добавляем к токенам
        if token not in stopwords and len(token) > 3: 
            tokens.append(token)
    return tokens

In [36]:
def count_frequencies(tokens):
    """
    Фнукиця для построения частотного списка.
    tokens: список (list) токенов.
    """
    #Частотный список с абсолютными частотами
    c = Counter(tokens).most_common()
    #Относительные частоты
    c_norm = [(el[0], round(el[1]/len(tokens), 3)) for el in c]
    print(c[:20])
    print(c_norm[:20])

In [37]:
#Предобрабатываем тексты
with open('./Texts/all_irish_lemmas.txt') as f:
    i_text = f.read()
with open('./Texts/all_bieber_lemmas.txt') as f:
    b_text = f.read()
b_tokens = preprocess(b_text)
i_tokens = preprocess(i_text)

In [38]:
#Строим частотный спиоск по песням Бибера
count_frequencies(b_tokens)

[('baby', 59), ('girl', 42), ('love', 40), ('time', 29), ('like', 23), ('tell', 22), ('never', 18), ('know', 14), ('heart', 13), ('always', 10), ('boyfriend', 10), ('want', 9), ('yeah', 9), ('mine', 9), ('world', 7), ('keep', 7), ('could', 7), ('think', 6), ('fight', 6), ('plus', 6)]
[('baby', 0.094), ('girl', 0.067), ('love', 0.063), ('time', 0.046), ('like', 0.037), ('tell', 0.035), ('never', 0.029), ('know', 0.022), ('heart', 0.021), ('always', 0.016), ('boyfriend', 0.016), ('want', 0.014), ('yeah', 0.014), ('mine', 0.014), ('world', 0.011), ('keep', 0.011), ('could', 0.011), ('think', 0.01), ('fight', 0.01), ('plus', 0.01)]


In [39]:
#Строим частотный список по ирландским песням
count_frequencies(i_tokens)

[('never', 14), ('take', 13), ('sally', 12), ('money', 10), ('wild', 8), ('brown', 8), ('rover', 7), ('come', 7), ('play', 6), ('spend', 6), ('ring', 5), ('home', 5), ('love', 5), ('pretty', 5), ('jenny', 5), ('tell', 4), ('delight', 4), ('bell', 4), ('snow', 4), ('would', 4)]
[('never', 0.03), ('take', 0.027), ('sally', 0.025), ('money', 0.021), ('wild', 0.017), ('brown', 0.017), ('rover', 0.015), ('come', 0.015), ('play', 0.013), ('spend', 0.013), ('ring', 0.011), ('home', 0.011), ('love', 0.011), ('pretty', 0.011), ('jenny', 0.011), ('tell', 0.008), ('delight', 0.008), ('bell', 0.008), ('snow', 0.008), ('would', 0.008)]


In [40]:
# RAKE algorithm

def rake(text):
# text = b_text+''
    #Разбиваем текст на токены и строим список уникальных слов
    tokens = preprocess(text)
    vocabulary = list(set(tokens))

    #Делаем разбиение по стоп-словам
    text = ' '.join(re.split(r'\W+', text))
    for w in stopwords:
        text = re.sub(r'\b'+w+r'\b', 'SPLIT', text)
        
    chunks = re.split(r'(?:SPLIT )+', text)
    chunks = [el.strip().split() for el in chunks]

    #Считаем degrees
    degrees = {}
    for word in vocabulary:
        word_degree = 0
        for chunk in chunks:
            if word in chunk:
                word_degree += len(chunk)
        degrees[word] = word_degree

    #Считаем frequencies
    frequencies = Counter(tokens)

    #Считаем RAKE scores
    rake_scores = {}
    for word in vocabulary:
        score = degrees[word]/frequencies[word]
        rake_scores[word] = score
    #Сортируем от большего к меньшему
    s = sorted(rake_scores.items(), key=lambda x: x[1], reverse=True)
    print('RAKE scores:\n', s[:30])
    print('Обычные частоты:\n', frequencies.most_common(30))

In [41]:
rake(b_text)

RAKE scores:
 [('timeas', 7.0), ('number', 6.0), ('camera', 6.0), ('issue', 6.0), ('makin', 6.0), ('mine', 5.777777777777778), ('climbin', 5.0), ('till', 5.0), ('gonna', 5.0), ('like', 4.913043478260869), ('yeah', 4.777777777777778), ('stomach', 4.0), ('lightyear', 4.0), ('billion', 4.0), ('pressure', 4.0), ('yeahh', 4.0), ('buzz', 4.0), ('seven', 4.0), ('snow', 4.0), ('blow', 4.0), ('item', 4.0), ('burr', 4.0), ('goif', 4.0), ('second', 4.0), ('people', 4.0), ('every', 4.0), ('across', 4.0), ('playin', 4.0), ('quit', 4.0), ('soldier', 4.0)]
Обычные частоты:
 [('baby', 59), ('girl', 42), ('love', 40), ('time', 29), ('like', 23), ('tell', 22), ('never', 18), ('know', 14), ('heart', 13), ('always', 10), ('boyfriend', 10), ('want', 9), ('yeah', 9), ('mine', 9), ('world', 7), ('keep', 7), ('could', 7), ('think', 6), ('fight', 6), ('plus', 6), ('long', 6), ('need', 6), ('ever', 5), ('make', 5), ('girlfriend', 5), ('gentleman', 4), ('swag', 4), ('cause', 4), ('first', 4), ('give', 4)]


In [42]:
rake(i_text)

RAKE scores:
 [('three', 6.0), ('please', 6.0), ('always', 5.0), ('nation', 5.0), ('still', 5.0), ('murray', 5.0), ('teeth', 5.0), ('mulatto', 5.0), ('musha', 5.0), ('dilly', 5.0), ('deceiver', 5.0), ('goose', 5.0), ('aboard', 5.0), ('bedford', 5.0), ('jamaica', 5.0), ('dumma', 5.0), ('dally', 5.0), ('bold', 4.5), ('seafaring', 4.0), ('mountain', 4.0), ('fair', 4.0), ('famed', 4.0), ('kerry', 4.0), ('maid', 4.0), ('whaler', 4.0), ('bound', 4.0), ('live', 4.0), ('instead', 4.0), ('roll', 3.6666666666666665), ('courtin', 3.5)]
Обычные частоты:
 [('never', 14), ('take', 13), ('sally', 12), ('money', 10), ('wild', 8), ('brown', 8), ('rover', 7), ('come', 7), ('play', 6), ('spend', 6), ('ring', 5), ('home', 5), ('love', 5), ('pretty', 5), ('jenny', 5), ('tell', 4), ('delight', 4), ('bell', 4), ('snow', 4), ('would', 4), ('sure', 4), ('bright', 3), ('white', 3), ('could', 3), ('morning', 3), ('produce', 3), ('captain', 3), ('first', 3), ('ship', 3), ('make', 3)]
